# Create the Final Edit with FFMPEG

This uses a cutlist JSON file, which must contain an array of cuts to making, include the starting and editing audio transition duration.

Each transition duration is different, since some clips may have longer pauses between words than others.

```json
[
    {
        "start_cut": 24.104,
        "start_transition": 0.9499999999999993,
        "end_cut": 157.8395,
        "end_transition": 1.679000000000002
    },
    ...
]


In [ ]:
import json
import subprocess

VIDEO_ID = 'gYXAulePuXY'
SOURCE_VIDEO = f"content/{VIDEO_ID}.f399.mp4"
SOURCE_AUDIO = f"content/{VIDEO_ID}.f251.webm"
OUTPUT_VIDEO = f"content/{VIDEO_ID}-edit.mp4"

# Load the cutlist from the previous step
cutlist_list = f"content/{VIDEO_ID}-cutlist.json"
with open(cutlist_list, 'r') as f:
    segments = json.load(f)

video_files = []
audio_files = []
num_clips = len(segments)

# Cut video and audio segments
for i, segment in enumerate(segments):
    video_output = f"content/{VIDEO_ID}-video_segment{i+1}.mp4"
    audio_output = f"content/{VIDEO_ID}-audio_segment{i+1}.m4a"
    video_files.append(video_output)
    audio_files.append(audio_output)
    
    # Cut video
    subprocess.run([
        "ffmpeg", "-y", "-i", SOURCE_VIDEO, "-ss", str(segment['start_cut']), "-to", str(segment['end_cut']),
        "-c:v", "libx264", "-an", "-preset", "fast", video_output
    ])
    
    # Cut audio
    start_transition = min(segment.get('start_transition', 0), segments[i - 1].get('end_transition', 0)) if i > 0 else 0
    end_transition = min(segment.get('end_transition', 0), segments[i + 1].get('start_transition', 0)) if i < (num_clips - 1) else 0
    start_offset = start_transition / 2
    end_offset = end_transition / 2
    subprocess.run([
        "ffmpeg", "-y", "-i", SOURCE_AUDIO, "-ss", format(segment['start_cut'] - start_offset, '.2f'), "-to", format(segment['end_cut'] + end_offset, '.2f'),
        "-c:a", "aac", audio_output
    ])
    
exit

# Prepare to concatenate, crossfade, and mux
input_args = []
for v in video_files:
    input_args.extend(["-i", v])
for a in audio_files:
    input_args.extend(["-i", a])

# Build filtergraph
video_filter = f"{'[' + ':v]['.join(str(i) for i in range(num_clips)) + ':v]'}" \
              f"concat=n={num_clips}:v=1:a=0[vout]"
audio_filter = ""
for i in range(num_clips - 1):
    in1 = f"{i+1 if i > 0 else ''}tmp" if i > 0 else f"{num_clips}:a"
    in2 = f"{num_clips+i+1}:a"
    out = f"{i+2}tmp" if i < num_clips - 2 else "aout"
    transition_duration = format(min(segments[i].get('end_transition', 0), segments[i + 1].get('start_transition', 0)), '.2f')
    print(segments[i].get('end_transition'), transition_duration)
    audio_filter += f"[{in1}][{in2}]acrossfade=d={transition_duration}:c1=tri:c2=tri[{out}];"
audio_filter = audio_filter.rstrip(";")
filter_complex = f"{video_filter};{audio_filter}"

# Build the ffmpeg command
command = [
    "ffmpeg", "-y", *input_args,
    "-filter_complex", filter_complex,
    "-map", "[vout]", "-map", "[aout]",
    "-c:v", "libx264", "-c:a", "aac", "-preset", "fast", OUTPUT_VIDEO
]

# Debug the command
print("Running command:", " ".join(command))

# Execute the command
try:
    subprocess.run(command, check=True, text=True, capture_output=True)
    print("FFmpeg command executed successfully")
except subprocess.CalledProcessError as e:
    print("FFmpeg failed with error:")
    print(e.stderr)

